In [ ]:
using GLPKMathProgInterface

In [ ]:
using JuMP

In [ ]:
using DataFrames

In [ ]:
#=
Variables for solving the problem (change these)
=#
# num_lineups is the total number of lineups
num_lineups = 100

# num_overlap is the maximum overlap of players between the lineups that you create
num_overlap = 7

# path_skaters is a string that gives the path to the csv file with the skaters information (see example file for suggested format)
path_skaters = "example_skaters.csv"

# path_goalies is a string that gives the path to the csv file with the goalies information (see example file for suggested format)
path_goalies = "example_goalies.csv"

# path_to_output is a string that gives the path to the csv file that will give the outputted results
path_to_output= "output.csv"

In [ ]:
function one_lineup_no_stacking(skaters, goalies, lineups, num_overlap, num_skaters, num_goalies, centers, wingers, defenders, num_teams, skaters_teams, goalie_opponents, team_lines, num_lines, P1_info)
    m = Model(solver=GLPKSolverMIP())

    # Variable for skaters in lineup.
    @variable(m, skaters_lineup[i=1:num_skaters], Bin)

    # Variable for goalie in lineup.
    @variable(m, goalies_lineup[i=1:num_goalies], Bin)

    
    # One goalie constraint
    @constraint(m, sum(goalies_lineup[i], i=1:num_goalies) == 1)

    # Eight Skaters constraint
    @constraint(m, sum(skaters_lineup[i], i=1:num_skaters) == 8)

    # between 2 and 3 centers
    @constraint(m, sum(centers[i]*skaters_lineup[i], i=1:num_skaters) <= 3)
    @constraint(m, 2 <= sum(centers[i]*skaters_lineup[i], i=1:num_skaters))

    # between 3 and 4 wingers
    @constraint(m, sum(wingers[i]*skaters_lineup[i], i=1:num_skaters) <= 4)
    @constraint(m, 3<=sum(wingers[i]*skaters_lineup[i], i=1:num_skaters))

    # between 2 and 3 defenders
    @constraint(m, 2 <= sum(defenders[i]*skaters_lineup[i], i=1:num_skaters))
    @constraint(m, sum(defenders[i]*skaters_lineup[i], i=1:num_skaters) <= 3)

    # Financial Constraint
    @constraint(m, sum(skaters[i,:Salary]*skaters_lineup[i], i=1:num_skaters) + sum(goalies[i,:Salary]*goalies_lineup[i], i=1:num_goalies) <= 50000)

    # at least 3 different teams for the 8 skaters constraints
    @variable(m, used_team[i=1:num_teams], Bin)
    @constraint(m, constr[i=1:num_teams], used_team[i] <= sum(skaters_teams[t, i]*skaters_lineup[t], t=1:num_skaters))
    @constraint(m, sum(used_team[i], i=1:num_teams) >= 3)

    # Overlap Constraint
    @constraint(m, constr[i=1:size(lineups)[2]], sum(lineups[j,i]*skaters_lineup[j], j=1:num_skaters) + sum(lineups[num_skaters+j,i]*goalies_lineup[j], j=1:num_goalies) <= num_overlap)


    # Objective
    @objective(m, Max, sum(skaters[i,:Projection]*skaters_lineup[i], i=1:num_skaters) + sum(goalies[i,:Projection]*goalies_lineup[i], i=1:num_goalies))


    # Solve the integer programming problem
    println("Solving Problem...")
    print("\n")
    status = solve(m);


    # Puts the output of one lineup into a format that will be used later
    if status==:Optimal
        skaters_lineup_copy = Array{Int64, 0}
        for i=1:num_skaters
            if getValue(skaters_lineup[i]) >= 0.9 && getValue(skaters_lineup[i]) <= 1.1
                skaters_lineup_copy = vcat(skaters_lineup_copy, fill(1,1))
            else
                skaters_lineup_copy = vcat(skaters_lineup_copy, fill(0,1))
            end
        end
        for i=1:num_goalies
            if getValue(goalies_lineup[i]) >= 0.9 && getValue(goalies_lineup[i]) <= 1.1
                skaters_lineup_copy = vcat(skaters_lineup_copy, fill(1,1))
            else
                skaters_lineup_copy = vcat(skaters_lineup_copy, fill(0,1))
            end
        end
        return(skaters_lineup_copy)
    end
end



In [ ]:
one_lineup_no_stacking

In [ ]:
function create_lineups(num_lineups, num_overlap, path_skaters, path_goalies, formulation, path_to_output)
    #=
    num_lineups is an integer that is the number of lineups
    num_overlap is an integer that gives the overlap between each lineup
    path_skaters is a string that gives the path to the skaters csv file
    path_goalies is a string that gives the path to the goalies csv file
    formulation is the type of formulation you would like to use (for instance one_lineup_Type_1, one_lineup_Type_2, etc.)
    path_to_output is a string where the final csv file with your lineups will be
    =#


    # Load information for skaters table
    skaters = readtable(path_skaters)

    # Load information for goalies table
    goalies = readtable(path_goalies)

    # Number of skaters
    num_skaters = size(skaters)[1]

    # Number of goalies
    num_goalies = size(goalies)[1]

    # wingers stores the information on which players are wings
    wingers = Array{Int64, 0}

    # centers stores the information on which players are centers
    centers = Array{Int64, 0}

    # defenders stores the information on which players are defenders
    defenders = Array{Int64, 0}

    #=
    Process the position information in the skaters file to populate the wingers,
    centers, and defenders with the corresponding correct information
    =#
    for i =1:num_skaters
        if skaters[i,:Position] == "LW" || skaters[i,:Position] == "RW" || skaters[i,:Position] == "W"
            wingers=vcat(wingers,fill(1,1))
            centers=vcat(centers,fill(0,1))
            defenders=vcat(defenders,fill(0,1))
        elseif skaters[i,:Position] == "C"
            wingers=vcat(wingers,fill(0,1))
            centers=vcat(centers,fill(1,1))
            defenders=vcat(defenders,fill(0,1))
        elseif skaters[i,:Position] == "D" || skaters[i,:Position] == "LD" || skaters[i,:Position] == "RD"
            wingers=vcat(wingers,fill(0,1))
            centers=vcat(centers,fill(0,1))
            defenders=vcat(defenders,fill(1,1))
        else
            wingers=vcat(wingers,fill(0,1))
            centers=vcat(centers,fill(0,1))
            defenders=vcat(defenders,fill(1,1))
        end
    end


    # A forward is either a center or a winger
    forwards = centers+wingers



    # Create team indicators from the information in the skaters file
    teams = unique(skaters[:Team])

    # Total number of teams
    num_teams = size(teams)[1]

    # player_info stores information on which team each player is on
    player_info = zeros(Int, size(teams)[1])

    # Populate player_info with the corresponding information
    for j=1:size(teams)[1]
        if skaters[1, :Team] == teams[j]
            player_info[j] =1
        end
    end
    skaters_teams = player_info'


    for i=2:num_skaters
        player_info = zeros(Int, size(teams)[1])
        for j=1:size(teams)[1]
            if skaters[i, :Team] == teams[j]
                player_info[j] =1
            end
        end
        skaters_teams = vcat(skaters_teams, player_info')
    end



    # Create goalie identifiers so you know who they are playing
    opponents = goalies[:Opponent]
    goalie_teams = goalies[:Team]
    goalie_opponents=[]
    for num = 1:size(teams)[1]
        if opponents[1] == teams[num]
            goalie_opponents = skaters_teams[:, num]
        end
    end
    for num = 2:size(opponents)[1]
        for num_2 = 1:size(teams)[1]
            if opponents[num] == teams[num_2]
                goalie_opponents = hcat(goalie_opponents, skaters_teams[:,num_2])
            end
        end
    end




    # Create line indicators so you know which players are on which lines
    L1_info = zeros(Int, num_skaters)
    L2_info = zeros(Int, num_skaters)
    L3_info = zeros(Int, num_skaters)
    L4_info = zeros(Int, num_skaters)
    for num=1:size(skaters)[1]
        if skaters[:Team][num] == teams[1]
            if skaters[:Line][num] == "1"
                L1_info[num] = 1
            elseif skaters[:Line][num] == "2"
                L2_info[num] = 1
            elseif skaters[:Line][num] == "3"
                L3_info[num] = 1
            elseif skaters[:Line][num] == "4"
                L4_info[num] = 1
            end
        end
    end
    team_lines = hcat(L1_info, L2_info, L3_info, L4_info)


    for num2 = 2:size(teams)[1]
        L1_info = zeros(Int, num_skaters)
        L2_info = zeros(Int, num_skaters)
        L3_info = zeros(Int, num_skaters)
        L4_info = zeros(Int, num_skaters)
        for num=1:size(skaters)[1]
            if skaters[:Team][num] == teams[num2]
                if skaters[:Line][num] == "1"
                    L1_info[num] = 1
                elseif skaters[:Line][num] == "2"
                    L2_info[num] = 1
                elseif skaters[:Line][num] == "3"
                    L3_info[num] = 1
                elseif skaters[:Line][num] == "4"
                    L4_info[num] = 1
                end
            end
        end
        team_lines = hcat(team_lines, L1_info, L2_info, L3_info, L4_info)
    end
    num_lines = size(team_lines)[2]




    # Create power play indicators
    PP_info = zeros(Int, num_skaters)
    for num=1:size(skaters)[1]
        if skaters[:Team][num]==teams[1]
            if skaters[:Power_Play][num] == "1"
                PP_info[num] = 1
            end
        end
    end

    P1_info = PP_info

    for num2=2:size(teams)[1]
        PP_info = zeros(Int, num_skaters)
        for num=1:size(skaters)[1]
            if skaters[:Team][num] == teams[num2]
                if skaters[:Power_Play][num] == "1"
                    PP_info[num]=1
                end
            end
        end
        P1_info = hcat(P1_info, PP_info)
    end


    # Lineups using formulation as the stacking type
    the_lineup= formulation(skaters, goalies, hcat(zeros(Int, num_skaters + num_goalies), zeros(Int, num_skaters + num_goalies)), num_overlap, num_skaters, num_goalies, centers, wingers, defenders, num_teams, skaters_teams, goalie_opponents, team_lines, num_lines, P1_info)
    the_lineup2 = formulation(skaters, goalies, hcat(the_lineup, zeros(Int, num_skaters + num_goalies)), num_overlap, num_skaters, num_goalies, centers, wingers, defenders, num_teams, skaters_teams, goalie_opponents, team_lines, num_lines, P1_info)
    tracer = hcat(the_lineup, the_lineup2)
    for i=1:(num_lineups-2)
        try
            thelineup=formulation(skaters, goalies, tracer, num_overlap, num_skaters, num_goalies, centers, wingers, defenders, num_teams, skaters_teams, goalie_opponents, team_lines, num_lines, P1_info)
            tracer = hcat(tracer,thelineup)
        catch
            break
        end
    end


    # Create the output csv file
    lineup2 = ""
    for j = 1:size(tracer)[2]
        lineup = ["" "" "" "" "" "" "" "" ""]
        for i =1:num_skaters
            if tracer[i,j] == 1
                if centers[i]==1
                    if lineup[1]==""
                        lineup[1] = string(skaters[i,1], " ", skaters[i,2])
                    elseif lineup[2]==""
                        lineup[2] = string(skaters[i,1], " ", skaters[i,2])
                    elseif lineup[9] ==""
                        lineup[9] = string(skaters[i,1], " ", skaters[i,2])
                    end
                elseif wingers[i] == 1
                    if lineup[3] == ""
                        lineup[3] = string(skaters[i,1], " ", skaters[i,2])
                    elseif lineup[4] == ""
                        lineup[4] = string(skaters[i,1], " ", skaters[i,2])
                    elseif lineup[5] == ""
                        lineup[5] = string(skaters[i,1], " ", skaters[i,2])
                    elseif lineup[9] == ""
                        lineup[9] = string(skaters[i,1], " ", skaters[i,2])
                    end
                elseif defenders[i]==1
                    if lineup[6] == ""
                        lineup[6] = string(skaters[i,1], " ", skaters[i,2])
                    elseif lineup[7] ==""
                        lineup[7] = string(skaters[i,1], " ", skaters[i,2])
                    elseif lineup[9] == ""
                        lineup[9] = string(skaters[i,1], " ", skaters[i,2])
                    end
                end
            end
        end
        for i =1:num_goalies
            if tracer[num_skaters+i,j] == 1
                lineup[8] = string(goalies[i,1], " ", goalies[i,2])
            end
        end
        for name in lineup
            lineup2 = string(lineup2, name, ",")
        end
        lineup2 = chop(lineup2)
        lineup2 = string(lineup2, """

        """)
    end
    outfile = open(path_to_output, "w")
    write(outfile, lineup2)
    close(outfile)
end




In [ ]:
formulation=one_lineup_no_stacking

In [ ]:
create_lineups(num_lineups, num_overlap, path_skaters, path_goalies, formulation, path_to_output)